In [1]:
import re
from Bio import SeqIO

In [2]:
def show_stats(filename):
    lengths = sorted(map(len, SeqIO.parse(filename, 'fasta')), reverse=True)
    n = sum(lengths)
    csum = 0
    for rec_len in lengths:
        csum += rec_len
        if csum * 2 >= n:
            n50 = rec_len
            break
    print(f'Stats for {filename}')
    print(f' - Number of records: {len(lengths)}')
    print(f' - Total length: {n}')
    print(f' - Longest record: {lengths[0]}')
    print(f' - N50: {n50}')
    
def extract_longest(filename):
    return max(SeqIO.parse(filename, 'fasta'), key=len)

def show_gap_stats(scaffold, desc):
    gaps = re.findall(r'N+', str(scaffold.seq))
    gap_count = len(gaps)
    total_length = sum(map(len, gaps))
    print(f'Gap stats for {desc}')
    print(f' - Gap count: {gap_count}')
    print(f' - Total gap length: {total_length}')

In [3]:
show_stats('data/contigs.fasta')
print()
show_stats('data/scaffolds_noClose.fasta')

Stats for data/contigs.fasta
 - Number of records: 607
 - Total length: 3924917
 - Longest record: 179307
 - N50: 53980

Stats for data/scaffolds_noClose.fasta
 - Number of records: 70
 - Total length: 3873759
 - Longest record: 3831941
 - N50: 3831941


In [4]:
longest = extract_longest('data/scaffolds_noClose.fasta')
longest_closed = extract_longest('data/scaffolds.fasta')
SeqIO.write([longest], 'data/longest_noClose.fasta', 'fasta')
SeqIO.write([longest_closed], 'data/longest.fasta', 'fasta')

1

In [5]:
show_gap_stats(longest, 'the longest scaffold')
print()
show_gap_stats(longest_closed, 'the longest scaffold (with closed gaps)')

Gap stats for the longest scaffold
 - Gap count: 63
 - Total gap length: 5979

Gap stats for the longest scaffold (with closed gaps)
 - Gap count: 12
 - Total gap length: 1312
